# Task-1
### NonCNN Network in MNIST

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

## 1. Define Network

In [2]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 2. Define training loop

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx , len(train_loader),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## 3. Model parameters

In [4]:
batch_size = 512
epochs = 1
lr = 0.01

seed = 1
device='cuda'

## 4. Define model

In [5]:
model = MLPNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

## 5. Build train / test dataloaders

In [6]:
dataset_train = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

dataset_test = datasets.MNIST('./data', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## 6. Training

In [7]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/118 (0%)]	Loss: 2.302011
Train Epoch: 1 [10/118 (8%)]	Loss: 0.378523
Train Epoch: 1 [20/118 (17%)]	Loss: 0.327173
Train Epoch: 1 [30/118 (25%)]	Loss: 0.247909
Train Epoch: 1 [40/118 (34%)]	Loss: 0.260601
Train Epoch: 1 [50/118 (42%)]	Loss: 0.167711
Train Epoch: 1 [60/118 (51%)]	Loss: 0.147566
Train Epoch: 1 [70/118 (59%)]	Loss: 0.192332
Train Epoch: 1 [80/118 (68%)]	Loss: 0.147985
Train Epoch: 1 [90/118 (76%)]	Loss: 0.212116
Train Epoch: 1 [100/118 (85%)]	Loss: 0.102416
Train Epoch: 1 [110/118 (93%)]	Loss: 0.097468

Test set: Average loss: 0.0003, Accuracy: 9564/10000 (95.64%)



## 7. Results

https://docs.google.com/spreadsheets/d/1lxPS6HTb5KDQu8ZirfUOdTG-Fhj2WfyELYcflMTCrws/edit?usp=sharing

|  optimizer | epochs_max | layers | batch_size | activation | best_acc |
| --- | --- | --- | --- | --- | --- |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 8 | relu | 90,00 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 16 | relu | 95,26 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 32 | relu | 96,72 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 64 | relu | 96,92 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 128 | relu | 97,28 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 256 | relu | 97,18 |
|  ADAM (0.01) | 10 | 3 (512, 256, 10) | 512 | relu | 97,36 |
|  SGD(0.01) | 10 | 3 (512, 256, 10) | 512 | relu | 97,68 |
|  SGD(0.01) | 25 | 3 (512, 256, 10) | 512 | relu | 98,00 |
|  SGD(0.001) start from exp000 | 25 | 3 (512, 256, 10) | 512 | relu | 98,45 |
|  SGD(0.0003) start from exp001 | 25 | 3 (512, 256, 10) | 512 | relu | **98,51** |
|   |  |  |  |  |  |
|  SGD(0.01) | 25 | 3 (512, 256, 10) | 512 | sigmoid | 98,00 |
|  SGD(0.001) from exp002 | 25 | 3 (512, 256, 10) | 512 | sigmoid | 98,40 |
|   |  |  |  |  |  |
|  ADAM(0.01) | 25 | 9(512,1024,2048,1024,512,258,128,64,10) | 512 | relu | 11,00 |
|  ADAM(0.01) | 25 | 5(512,512,512,128,10) | 512 | relu | 97,80 |
|  SGD(0.001) from exp003 | 25 | 5(512,512,512,128,10) | 512 | relu | 98,16 |